### Jupyter

In [1]:
from scipy.io import wavfile
from scipy.fftpack import fft, ifft, rfft
from scipy import signal

import matplotlib.pyplot as plt
import IPython.display as ipd


In [2]:
def analysis(wav_data, N, hop_a = None):
    assert(len(wav_data.shape) == 1)
    if hop_a is None:
        hop_a = N // 4
    # padding data to ensure seamless fft computation
    pad_len = hop_a - (wav_data.shape[0] - N) % hop_a
    wav_data = np.pad(wav_data, (0, pad_len), mode='constant')
    hann_window = signal.hann(N)
    
    new_shape = (N, 1 + (wav_data.shape[0] - N) // hop_a)
    spectra = np.zeros(new_shape, dtype=np.complex128)
    for i in range(new_shape[1]):
        windowed_frame = wav_data[hop_a * i:hop_a * i + N] * hann_window
        spectra[:, i] = fft(windowed_frame) # Eq 3.1
    return spectra

In [3]:
def processing(spectra, sampling_freq, semitones = 0, hop_a=None):
    N = spectra.shape[0]
    if hop_a is None:
        hop_a = N // 4
    bin_freq = np.fft.fftfreq(N, 1/(2 * np.pi))
    dt_a = hop_a / sampling_freq # pre Eq. 3.2
    scaling_factor = 2**(semitones / 12)
    hop_s = int(scaling_factor * hop_a)
    dt_s = hop_s / sampling_freq
    
    phases = np.zeros_like(spectra, dtype=np.float64)
    phases[:, 0] = np.angle(spectra[:, 0])
    for i in range(1, spectra.shape[1]):
        dphase_a = np.angle(spectra[:, i]) - np.angle(spectra[:, i - 1])
        freq_deviation = dphase_a/dt_a - bin_freq # Eq. 3.3
        wrapped_freq_deviation = np.mod(freq_deviation + np.pi, 2 * np.pi) - np.pi # Eq. 3.4
        true_freq = bin_freq + wrapped_freq_deviation # Eq. 3.5
        
        phases[:, i] = phases[:, i - 1] + dt_s * true_freq # Eq. 3.6
    
    new_spectra = np.abs(spectra) * np.exp(1j * phases)
    return new_spectra, hop_s

In [4]:
def synthesis(new_spectra, hop_s):
    N = new_spectra.shape[0]
    L = new_spectra.shape[1]
    hann_window = signal.hann(N)
    
    waves = np.zeros_like(new_spectra, dtype=np.float64)
    for i in range(L):
        waves[:, i] = ifft(new_spectra[:, i]).real * hann_window # Eq. 3.8
    
    stretched_wave = np.zeros(N + (L - 1) * hop_s, dtype=np.complex128)
    for i in range(L):
        stretched_wave[i * hop_s:i * hop_s + N] += waves[:, i]
    return stretched_wave
    

In [5]:
fs, data = wavfile.read('data/terminator2.wav')
ipd.Audio(data, rate=fs)


In [6]:
N = 1024
hop_a = N // 4
semitones = -2
spectra = analysis(data, N, hop_a)

new_spectra, hop_s = processing(spectra, fs, semitones)
stretched_wave = synthesis(new_spectra, hop_s).real

# scaling_factor = 2**(semitones / 12)
# hop_s = int(scaling_factor * hop_a) 
# stretched_wave = synthesis(spectra, hop_s)


xp = np.arange(len(stretched_wave)) + 1
x = np.linspace(0, len(stretched_wave), len(data) + 1)[1:]
result_wave = np.interp(x, xp, stretched_wave)
ipd.Audio(result_wave, rate=fs)

In [13]:
wavfile.write('terminator_out.wav', fs, result_wave.astype(np.int16))